<a href="https://colab.research.google.com/github/mascDriver/data-science/blob/main/GEX637_Trabalho_4(refatorando)_%E2%80%93_Busca_Gulosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from scipy.spatial import distance_matrix

instances = {
    "dj38.tsp": "Djibouti",
    "qa194.tsp": "Qatar",
    "uy734.tsp": "Uruguay",
    "wi29.tsp": "Western Sahara",
    "zi929.tsp": "Zimbabwe",
}

path = '/content/drive/MyDrive/GEX637/'

class Tsp():
  def read_tsp(self, filename):
    with open(filename) as f:
        node_coord_start = None
        dimension = None
        lines = f.readlines()
        i = 0
        while not dimension or not node_coord_start:
            line = lines[i]
            if line.startswith('DIMENSION :') or line.startswith('DIMENSION:'):
                dimension = int(line.split()[-1])
                self.dimension = dimension
            if line.startswith('NODE_COORD_SECTION'):
                node_coord_start = i
            i = i+1
        f.seek(0)
        cities = pd.read_csv(
            f,
            skiprows=node_coord_start + 1,
            sep=' ',
            names=['city', 'x', 'y'],
            dtype={'city': str, 'x': np.float64, 'y': np.float64},
            header=None,
            nrows=dimension
        )
        self.cities = cities

  def matrix(self):
    self.matrix_distance = distance_matrix(self.cities[['x', 'y']].values, self.cities[['x', 'y']].values)

class GreedySearch():
  def __init__(self, matrix_distance, dimension) -> None:
      self.matrix_distance = matrix_distance
      self.dimension = dimension
      self.positive_inf = float('inf')
      self.selected_nodes = np.full(self.dimension, False)
      self.result = np.zeros((self.dimension, self.dimension), dtype=int)
      self.quality = 0

  def mst(self):
    # used here Prim's algorithm - minimum spanning tree

    elapsed_time_instance =  self.dimension * 0.06
    elapsed_time = 0
    t = time.perf_counter()

    while(elapsed_time < elapsed_time_instance):
      while(False in self.selected_nodes):
        minimum = self.positive_inf
        start = np.random.randint(self.dimension)
        end = np.random.randint(self.dimension)
        for i in range(self.dimension):
          if self.selected_nodes[i]:
            for j in range(self.dimension):
              if not self.selected_nodes[j] and self.matrix_distance[i][j]>0:
                if self.matrix_distance[i][j] < minimum:
                  minimum = self.matrix_distance[i][j]
                  start, end = i, j
        self.selected_nodes[end] = True

        if minimum == self.positive_inf:
          self.result[start][end] = 0
        else:
          self.result[start][end] = round(minimum)
          try:
            self.result_route = np.append([[start, end]], self.result_route, axis=0)
          except:
            self.result_route = np.array([[start, end]])
        self.result[end][start] = self.result[start][end]
      elapsed_time = time.perf_counter() - t

  def calculate_quality(self):
    temp_route = np.append(self.result_route, [self.result_route[0]], axis=0)
    for x, y in temp_route:
      self.quality += self.matrix_distance[x][y]
    

def main():
  resultados = []
  for filename in os.scandir(path):
    tsp = Tsp() # instancia a class responsavel por arquivos tsp
    tsp.read_tsp(filename) # extrai os dados necessarios para utilizar no cod
    tsp.matrix() # cria matriz de distancias euclidianas

    graph = GreedySearch(tsp.matrix_distance, tsp.dimension) # instancia busca gulosa
    quality = []
    time_execution = []
    for _ in range(10):
      t = time.perf_counter()
      graph.mst() # utiliza algoritmo arvore geradora minima para resolver problema
      graph.calculate_quality() # calcula qualidade encontrada no algoritmo anterior
      quality.append(graph.quality)
      time_execution.append(time.perf_counter() - t)
    resultados.append({
        'instancia': instances[filename.name],
        'autoria': 'Diogo.Baltazar',
        'algoritmo': 'BCGα',
        'q-medio': round(np.mean(quality)),
        'q-desvio': round(np.std(quality), 2),
        't-medio': round(np.mean(time_execution))
    })
  pd.DataFrame(resultados).to_csv('resultados.csv', index=False)
main()

In [3]:
pd.read_csv('resultados.csv')

,instancia,autoria,algoritmo,q-medio,q-desvio,t-medio
0,Djibouti,Diogo.Baltazar,BCGα,33699,17598.92,2
1,Qatar,Diogo.Baltazar,BCGα,45964,24003.72,12
2,Uruguay,Diogo.Baltazar,BCGα,386026,201595.33,47
3,Western Sahara,Diogo.Baltazar,BCGα,120038,62687.89,2
4,Zimbabwe,Diogo.Baltazar,BCGα,460651,240566.92,65
